In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
from tqdm.auto import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob

In [3]:
root = '/content/drive/MyDrive/Colab Notebooks/AF_pj/dataset/'
map_pm = pd.read_csv(root+'META/pmmap.csv')
map_aws = pd.read_csv(root+'META/awsmap.csv')

label_pm = list(map_pm['Location'])
label_aws = list(map_aws['Location'])
display(map_pm.head(2))
display(map_aws.head(2))
print(len(label_pm), label_pm)
print(len(label_aws), label_aws)

,Location,Latitude,Longitude,Description
0,아름동,36.512252,127.246789,세종특별자치시 보듬3로 114 아름동커뮤니티센터 옥상 (아름동)
1,신흥동,36.592887,127.292550,세종 조치원읍 군청로 87-16(신흥동) 세종특별자치시 조치원청사 옥상


,Location,Latitude,Longitude,Description
0,오월드,36.2913,127.3959,대전광역시 중구사정공원로 70 오월드 내 플라워랜드
1,세천,36.3402,127.4938,대전광역시 동구세천동 63-1


17 ['아름동', '신흥동', '노은동', '문창동', '읍내동', '정림동', '공주', '논산', '대천2동', '독곶리', '동문동', '모종동', '신방동', '예산군', '이원면', '홍성읍', '성성동']
30 ['오월드', '세천', '장동', '세종고운', '세종금남', '세종연서', '세종전의', '북격렬비도', '호도', '대산', '홍북', '홍성죽도', '공주', '서천', '논산', '당진', '성거', '청양', '태안', '예산', '유구', '정안', '아산', '양화', '계룡', '춘장대', '대천항', '안도', '옹도', '정산']


In [8]:
near_pmaws = {loc_pm:None for loc_pm in label_pm}
for loc_pm in label_pm:
    lat_pm = map_pm.loc[map_pm['Location']==loc_pm, 'Latitude'].item()
    log_pm = map_pm.loc[map_pm['Location']==loc_pm, 'Longitude'].item()
    min_d = np.inf
    for loc_aws in label_aws:
        lat_aws = map_aws.loc[map_aws['Location']==loc_aws, 'Latitude'].item()
        log_aws = map_aws.loc[map_aws['Location']==loc_aws, 'Longitude'].item()
        distance = np.sqrt((lat_pm-lat_aws)**2 + (log_pm-log_aws)**2)
        if distance<min_d: 
            min_d = distance
            near_pmaws[loc_pm] = loc_aws
print(near_pmaws)

{'아름동': '세종고운', '신흥동': '세종연서', '노은동': '계룡', '문창동': '오월드', '읍내동': '장동', '정림동': '오월드', '공주': '공주', '논산': '논산', '대천2동': '대천항', '독곶리': '대산', '동문동': '태안', '모종동': '아산', '신방동': '성거', '예산군': '예산', '이원면': '태안', '홍성읍': '홍북', '성성동': '성거'}


In [9]:
train_dic = {}
for loc_pm in label_pm:
    pm_df = pd.read_csv(root+'TRAIN/'+loc_pm+'.csv')
    pm_df['Latitude'] = map_pm.loc[map_pm['Location']==loc_pm, 'Latitude'].item()
    pm_df['Longitude'] = map_pm.loc[map_pm['Location']==loc_pm, 'Longitude'].item()
    aws_df = pd.read_csv(root+'TRAIN_AWS/'+near_pmaws[loc_pm]+'.csv')
    train_dic[loc_pm] = pd.concat([pm_df, aws_df[['기온(°C)','풍향(deg)','풍속(m/s)','강수량(mm)','습도(%)']]],
                                  axis=1)
    train_dic[loc_pm].fillna(method='ffill', limit=10, inplace=True)
    train_dic[loc_pm].fillna(train_dic[loc_pm].mean(numeric_only=True), inplace=True)
    
train_dic['공주'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35064 entries, 0 to 35063
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   연도         35064 non-null  int64  
 1   일시         35064 non-null  object 
 2   측정소        35064 non-null  object 
 3   PM2.5      35064 non-null  float64
 4   Latitude   35064 non-null  float64
 5   Longitude  35064 non-null  float64
 6   기온(°C)     35064 non-null  float64
 7   풍향(deg)    35064 non-null  float64
 8   풍속(m/s)    35064 non-null  float64
 9   강수량(mm)    35064 non-null  float64
 10  습도(%)      35064 non-null  float64
dtypes: float64(8), int64(1), object(2)
memory usage: 2.9+ MB


In [15]:
test_dic = {}
for loc_pm in label_pm:
    pm_df = pd.read_csv(root+'TEST_INPUT/'+loc_pm+'.csv')
    pm_df['Latitude'] = map_pm.loc[map_pm['Location']==loc_pm, 'Latitude'].item()
    pm_df['Longitude'] = map_pm.loc[map_pm['Location']==loc_pm, 'Longitude'].item()
    aws_df = pd.read_csv(root+'TEST_AWS/'+near_pmaws[loc_pm]+'.csv')
    test_dic[loc_pm] = pd.concat([pm_df, aws_df[['기온(°C)','풍향(deg)','풍속(m/s)','강수량(mm)','습도(%)']]],
                                  axis=1)
    test_dic[loc_pm].fillna(method='ffill', limit=10, inplace=True)
    test_dic[loc_pm].fillna(test_dic[loc_pm].mean(numeric_only=True), inplace=True)
    
test_dic['아름동'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7728 entries, 0 to 7727
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   연도         7728 non-null   int64  
 1   일시         7728 non-null   object 
 2   측정소        7728 non-null   object 
 3   PM2.5      7728 non-null   float64
 4   Latitude   7728 non-null   float64
 5   Longitude  7728 non-null   float64
 6   기온(°C)     7728 non-null   float64
 7   풍향(deg)    7728 non-null   float64
 8   풍속(m/s)    7728 non-null   float64
 9   강수량(mm)    7728 non-null   float64
 10  습도(%)      7728 non-null   float64
dtypes: float64(8), int64(1), object(2)
memory usage: 664.2+ KB


In [16]:
aws_df = pd.read_csv(root+'TEST_AWS/'+'세종고운'+'.csv')
aws_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7728 entries, 0 to 7727
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   연도       7728 non-null   int64  
 1   일시       7728 non-null   object 
 2   지점       3120 non-null   object 
 3   기온(°C)   3120 non-null   float64
 4   풍향(deg)  3120 non-null   float64
 5   풍속(m/s)  3120 non-null   float64
 6   강수량(mm)  3120 non-null   float64
 7   습도(%)    3120 non-null   float64
dtypes: float64(5), int64(1), object(2)
memory usage: 483.1+ KB


In [ ]:
def ex_df(train_dic, location):
    ex = train_dic[location].loc[train_dic[location]['연도']==0].copy()
    ex.drop(columns=['연도','측정소'], inplace=True)
    ex['월'] = ex['일시'].apply(lambda x: int(x.split('-')[0]))
    ex.columns = [location+'_'+col for col in ex.columns]
    ex.rename(columns={location+'_일시':'일시', location+'_월':'월'}, inplace=True)
    return ex.reindex(columns=['일시','월']+ex.columns.tolist()[1:-1])
ex1 = ex_df(train_dic, '공주')
ex2 = ex_df(train_dic, '노은동')
# display(ex1.head(2)); display(ex2.head(2))
ex = pd.concat([ex1, ex2.drop(columns=['일시','월'])], axis=1)
display(ex.info())
# ex.head(3)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8760 entries, 0 to 8759
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   일시             8760 non-null   object 
 1   월              8760 non-null   int64  
 2   공주_PM2.5       8760 non-null   float64
 3   공주_Latitude    8760 non-null   float64
 4   공주_Longitude   8760 non-null   float64
 5   공주_기온(°C)      8760 non-null   float64
 6   공주_풍향(deg)     8760 non-null   float64
 7   공주_풍속(m/s)     8760 non-null   float64
 8   공주_강수량(mm)     8760 non-null   float64
 9   공주_습도(%)       8760 non-null   float64
 10  노은동_PM2.5      8760 non-null   float64
 11  노은동_Latitude   8760 non-null   float64
 12  노은동_Longitude  8760 non-null   float64
 13  노은동_기온(°C)     8760 non-null   float64
 14  노은동_풍향(deg)    8760 non-null   float64
 15  노은동_풍속(m/s)    8760 non-null   float64
 16  노은동_강수량(mm)    8760 non-null   float64
 17  노은동_습도(%)      8760 non-null   float64
dtypes: float

None

In [ ]:
# 0~1 사이로 스케일링
ex.iloc[:,1:] = ex.iloc[:,1:]/ex.iloc[:,1:].max()
ex.head(2)

,일시,월,공주_PM2.5,공주_Latitude,공주_Longitude,공주_기온(°C),공주_풍향(deg),공주_풍속(m/s),공주_강수량(mm),공주_습도(%),노은동_PM2.5,노은동_Latitude,노은동_Longitude,노은동_기온(°C),노은동_풍향(deg),노은동_풍속(m/s),노은동_강수량(mm),노은동_습도(%)
0,01-01 00:00,0.083333,0.104478,1.0,1.0,0.177134,0.201944,0.123288,0.0,0.838906,0.089655,1.0,1.0,0.158833,0.411062,0.188406,0.0,0.841785
1,01-01 01:00,0.083333,0.111940,1.0,1.0,0.180354,0.168611,0.164384,0.0,0.841945,0.124138,1.0,1.0,0.145867,0.148694,0.173913,0.0,0.865112


In [ ]:
en_window_s = 2*24
de_window_s = 3*24
x_data, y_data = [], []
for i in range(ex.shape[0] - en_window_s - de_window_s):
    x = ex.iloc[i:i+en_window_s, 1:].to_numpy()
    x_data.append(x)
    y = ex.iloc[i+en_window_s:i+en_window_s+de_window_s, 2::8]
    y_data.append(np.concatenate([np.zeros([1,y.shape[1]]), y], axis=0))
x_data, y_data = np.array(x_data), np.array(y_data)
x_data.shape, y_data.shape

((8640, 48, 17), (8640, 73, 2))

In [ ]:
num_valid = 1
x_train = x_data[:-num_valid-de_window_s]
y_train = y_data[:-num_valid-de_window_s]
x_valid = x_data[-num_valid:]
y_valid = y_data[-num_valid:]
x_train.shape, y_train.shape, x_valid.shape, y_valid.shape

((8567, 48, 17), (8567, 73, 2), (1, 48, 17), (1, 73, 2))

In [ ]:
class myDataset(Dataset):
    def __init__(self, en_input, de_input):
        self.en_input, self.de_input = en_input, de_input

    def __len__(self):
        return len(self.en_input)
    
    def __getitem__(self, idx):
        encoder_input = torch.tensor(self.en_input[idx], dtype=torch.float32)
        decoder_input = torch.tensor(self.de_input[idx], dtype=torch.float32)
        return {'encoder_input':encoder_input, 'decoder_input':decoder_input}

trainset = myDataset(x_train, y_train)
validset = myDataset(x_valid, y_valid)
len(trainset), len(validset)

(8567, 1)

In [ ]:
batch_s = 128
trainloader = DataLoader(trainset, batch_size=batch_s, num_workers=16, shuffle=True)
validloader = DataLoader(validset, batch_size=batch_s, num_workers=16, shuffle=False)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
batch = next(iter(trainloader))
batch['encoder_input'].shape, batch['decoder_input'].shape

(torch.Size([128, 48, 17]), torch.Size([128, 73, 2]))

In [ ]:
batch_s = 16; en_T = 48; en_C = 17; en_cols = 9
# batch_s, encoder timesteps, encoder channels, encoder columns(월, pm2.5, ..., 강수량)
en_hidden_dim = 128 # en_hidden_dim = de_hidden_dim = hidden_dim
n_layers = 3

en_input = torch.randn(batch_s, en_T, en_C, en_cols);  print('encoder input:', en_input.shape)
en_input = en_input.reshape(batch_s*en_T, en_C, en_cols); print('en input reshape:', en_input.shape)
output = nn.Conv1d(17, 17, 3)(en_input);   print('conv1d:', output.shape) # +selu
output = nn.Conv1d(17, 17, 3)(output);    print('conv1d:', output.shape)
output = nn.BatchNorm1d(17)(output);    print('bn:', output.shape) # +selu
output = nn.Conv1d(17,17, 3)(output);    print('conv1d:', output.shape)
output = nn.BatchNorm1d(17)(output);    print('bn:', output.shape) # +selu
output = nn.MaxPool1d(2,stride=2)(output); print('polling:', output.shape)
output = output.reshape(batch_s, en_T, -1); print('output reshape:', output.shape)
hidden = (torch.zeros(n_layers, batch_s, en_hidden_dim),
               torch.zeros(n_layers, batch_s, en_hidden_dim))
print('hidden', hidden[0].shape)
en_output, en_hidden = nn.LSTM(17, en_hidden_dim, n_layers, batch_first=True)(output, hidden)
print('en_output, en_hidden:', en_output.shape, en_hidden[0].shape)

encoder input: torch.Size([16, 48, 17, 9])
en input reshape: torch.Size([768, 17, 9])
conv1d: torch.Size([768, 17, 7])
conv1d: torch.Size([768, 17, 5])
bn: torch.Size([768, 17, 5])
conv1d: torch.Size([768, 17, 3])
bn: torch.Size([768, 17, 3])
polling: torch.Size([768, 17, 1])
output reshape: torch.Size([16, 48, 17])
hidden torch.Size([3, 16, 128])
en_output, en_hidden: torch.Size([16, 48, 128]) torch.Size([3, 16, 128])


In [ ]:
target_n = 17; de_hidden_dim = 128; n_layers = 3
decoder_input = torch.randn(batch_s, 73, target_n)
batch_s, t_len = decoder_input.size(0), decoder_input.size(1)
print(batch_s, t_len)

outputs = torch.zeros(batch_s, t_len-1, target_n)
print('outputs:', outputs.shape)
de_input = decoder_input[:,0]
print('de_input:', de_input.shape) # 64, 17
for t in range(1, t_len):
    if t==1: de_hidden = en_hidden
#     de_output, de_hidden = self.decoder(en_output, de_input, de_hidden)
    de_input = nn.Linear(target_n, en_hidden_dim)(de_input)#.view(batch_s, 1, -1) # 64,1,128
    # cont_v, att_w = self.attention(de_hidden, en_output)
    print('de_input',de_input.shape, de_input.unsqueeze(1).shape, de_input.view(batch_s,1,-1).shape)
    A = nn.Linear(128,128)(de_input.view(batch_s,1,-1))
    B = nn.Linear(128,128)(en_output)
    AB = A+B
    print('A,B,AB:', A.shape, B.shape, AB.shape)
    score = nn.Linear(128,1)(torch.tanh(AB))
    print('score:', score.shape)
    att_w = torch.softmax(score, axis=1)
    print('att_w:', att_w.shape)
    cont_v = att_w*en_output
    print('cont_v:', cont_v.shape)
    cont_v = torch.sum(cont_v, dim=1)
    print('total cont_v:', cont_v.shape)
    print('sum cont_v:', torch.sum(cont_v, dim=0).unsqueeze(0).shape)
    de_input = torch.cat([cont_v, de_input], dim=1)
    print('concat:', de_input.shape) # 64,256
    de_input = de_input.unsqueeze(0)
    print('de_input:', de_input.shape) # 1,64,256
    de_output, de_hidden = nn.LSTM(en_hidden_dim*2, de_hidden_dim, n_layers, batch_first=True)(de_input, de_hidden)
    print('de_output, h', de_output.shape, de_hidden[0].shape) # 1,64,128 / 3,64,128
    print('de_output.sum(0)', de_output.sum(0).shape) # 64,128
    pred = nn.Linear(de_hidden_dim, target_n)(de_output.sum(0))
    print('pred:', pred.shape)
    outputs[:,t-1] = pred
    # if training:
    #     de_input = decoder_input[:,t]
    # else:
    #     de_input = output
    if t==1: break

NameError: ignored

In [ ]:
16 73
outputs: torch.Size([16, 72, 17])
de_input: torch.Size([16, 17])
de_input torch.Size([16, 128]) torch.Size([16, 1, 128]) torch.Size([16, 1, 128])
A,B,AB: torch.Size([16, 1, 128]) torch.Size([16, 48, 128]) torch.Size([16, 48, 128])
score: torch.Size([16, 48, 1])
att_w: torch.Size([16, 48, 1])
cont_v: torch.Size([16, 48, 128])
total cont_v: torch.Size([16, 128])
sum cont_v: torch.Size([1, 128])
concat: torch.Size([16, 256])
de_input: torch.Size([1, 16, 256])
de_output, h torch.Size([1, 16, 128]) torch.Size([3, 16, 128])
de_output.sum(0) torch.Size([16, 128])
pred: torch.Size([16, 17])

In [ ]:
# train loop에서 init_hidden 호출?
class Encoder(nn.Module):
    def __init__(self, cnn_dims, input_dim, hidden_dim, n_layers):
        super().__init__()
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        self.cnn = nn.Sequential(nn.Conv1d(cnn_dims[0], cnn_dims[1], 1), # 
                                 nn.SELU(),
                                 nn.Conv1d(cnn_dims[1], cnn_dims[2], 1), # 
                                 nn.BatchNorm1d(cnn_dims[2]), nn.SELU(),
                                 nn.Conv1d(cnn_dims[2], cnn_dims[3], 1), # 
                                 nn.BatchNorm1d(cnn_dims[3]), nn.SELU(),
                                 nn.AvgPool1d(2, stride=1) #
                                )
        input_dim = int(cnn_dims[-1]/2)
        self.lstm = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first=True)
#         self.hidden = self.init_hidden()
        
    def forward(self, inputs): # inputs: batch_s, times, cols
        output = self.cnn(inputs)
        output, hidden = self.lstm(output, hidden)
#         output, self.hidden = self.lstm(output, self.hidden)
        return output, hidden
    
    def init_hidden(self, batch_s):
        return (torch.zeros(self.n_layers, batch_s, self.hidden_dim),
               torch.zeros(self.n_layers, batch_s, self.hidden_dim))

In [ ]:
## how to use attention torch -> 검색ㄱㄱ
class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.fc_hidden = nn.Linear(hidden_dim, hidden_dim)
        self.fc_encoder = nn.Linear(hidden_dim, hidden_dim)
        self.scoring = nn.Linear(hidden_dim, 1)
        
    def forward(self, hidden, en_output):
        score = self.scoring(torch.tanh(self.fc_hidden(hidden.unsqueeze(1)) + self.fc_encoder(en_output)))
        att_w = torch.softmax(score, axis=1)
        context_vec = att_w * en_output
        context_vec = torch.sum(context_vec, dim=1)
        # att_w = torch.softmax(score.squeeze(2), dim=-1)
        # context_vec = att_w.unsqueeze(2) * en_output
        # context_vec = context_vec.sum(1)
        return context_vec, att_w

In [ ]:
## attention decoder lstm batch_first -> 검색 ㄱㄱ
class Decoder(nn.Module):
    def __init__(self, de_input_s, en_hidden_dim, de_hidden_dim, output_dim,
                 n_layers, attention):
        super().__init__()
        self.n_layers = n_layers
        self.attention = attention
        
        self.layer = nn.Linear(de_input_s, en_hidden_dim)
        self.lstm = nn.LSTM(en_hidden_dim*2, de_hidden_dim, n_layers)
        self.fc = nn.Linear(de_hidden_dim, output_dim)
        
    def forward(self, en_output, de_input, hidden):
        de_input = self.layer(de_input)
        context_vec, att_w = self.attension(hidden, en_output)
        de_input = torch.cat([torch.sum(context_vec, dim=0), de_input], dim=1).unsqueeze(0)
        output, hidden = self.lstm(de_input, hidden)
        output = self.fc(output.sum(0))
        return output, hidden

    def init_hidden(self, batch_s):
        return (torch.zeros(self.n_layers, batch_s, self.hidden_dim),
               torch.zeros(self.n_layers, batch_s, self.hidden_dim))

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, attention):
        super().__init__()
        self.encoder, self.decoder = encoder, decoder
        self.attention = attention
        
    def forward(self, en_input, de_input, training=True):
        batch_s, t_len = de_input.size(0), de_input.size(1)
        
        outputs = torch.zeros(batch_s, t_len-1, self.decoder.output_dim)#.to(device)
        en_output, hidden = self.encoder(en_input)
        de_input = de_input[:,0]
        for t in range(0, t_len-1):
            output, hidden = self.decoder(en_output, de_input, hidden)
            outputs[:t] = output
            if training:
                de_input = de_input[:,t+1]
            else:
                de_input = output
        return outputs

In [ ]:
target_n = 2 #17 # 맞춰야하는 관측소 pm2.5 수
n_layers = 3 # lstm 레이어 층
hidden_dim = 128 # lstm 히든차원
# cnn_dims = [x_data.shape[-1], n1, n2, n3] # 칼럼 수, cnn 출력층 수 1,2,3
cnn_dims = [17, 12, 10, 6] # 칼럼 수, cnn 출력층 수 1,2,3

encoder = Encoder(cnn_dims, input_dim=None, hidden_dim=hidden_dim, n_layers=n_layers)
attention = Attention(hidden_dim=hidden_dim)
decoder = Decoder(de_input_s=target_n, en_hidden_dim=hidden_dim, de_hidden_dim=hidden_dim,
                  output_dim=target_n, n_layers=n_layers, attention=attention)
model = Seq2Seq(encoder, decoder, attention)
model#.to(device)

Seq2Seq(
  (encoder): Encoder(
    (cnn): Sequential(
      (0): Conv1d(17, 12, kernel_size=(1,), stride=(1,))
      (1): SELU()
      (2): Conv1d(12, 10, kernel_size=(1,), stride=(1,))
      (3): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (4): SELU()
      (5): Conv1d(10, 6, kernel_size=(1,), stride=(1,))
      (6): BatchNorm1d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (7): SELU()
      (8): AvgPool1d(kernel_size=(2,), stride=(1,), padding=(0,))
    )
    (lstm): LSTM(3, 128, num_layers=3, batch_first=True)
  )
  (decoder): Decoder(
    (attention): Attention(
      (W1): Linear(in_features=128, out_features=128, bias=True)
      (W2): Linear(in_features=128, out_features=128, bias=True)
      (V): Linear(in_features=128, out_features=1, bias=True)
    )
    (layer): Linear(in_features=2, out_features=128, bias=True)
    (lstm): LSTM(256, 128, num_layers=3)
    (fc): Linear(in_features=128, out_features=2, bias=

In [ ]:
## 다시 짤 것

In [ ]:
inputs = torch.randn(16, 7, 17);        print('input', inputs.shape)
output = nn.Conv1d(7, 30, 1)(inputs);   print('conv1d', output.shape)
output = nn.SELU()(output);             print('selu', output.shape)
output = nn.Conv1d(30,40,1)(output);    print('conv1d', output.shape)
output = nn.BatchNorm1d(40)(output);    print('bn', output.shape)
output = nn.SELU()(output);             print('selu', output.shape)
output = nn.Conv1d(40,20,1)(output);    print('conv1d', output.shape)
output = nn.BatchNorm1d(20)(output);    print('bn', output.shape)
output = nn.SELU()(output);             print('selu', output.shape)
output = nn.MaxPool1d(2,stride=1)(output);print('polling', output.shape)

input torch.Size([16, 7, 17])
conv1d torch.Size([16, 30, 17])
selu torch.Size([16, 30, 17])
conv1d torch.Size([16, 40, 17])
bn torch.Size([16, 40, 17])
selu torch.Size([16, 40, 17])
conv1d torch.Size([16, 20, 17])
bn torch.Size([16, 20, 17])
selu torch.Size([16, 20, 17])
polling torch.Size([16, 20, 16])


In [ ]:
class CNN_LSTM(nn.Module):
    def __init__(self):
        # # input(batch_s, cols_len, pms_len) -> 
        # # conv1d -> selu -> conv1d -> bn -> selu -> conv1d -> bn -> selu -> 
        # #     lstm -> fc -> sigmoid -> output
        super.__init__()
        # input shape: 16,7,17
        A,B,C = 30,40,20
        self.convblock = nn.Sequential(nn.Conv1d(7, A, 1), # 16,A,17
                                       nn.SELU(),
                                       nn.Conv1d(A, B, 1), # 16,B,17
                                       nn.BatchNorm1d(B), 
                                       nn.SELU(),
                                       nn.Conv1d(B, C, 1), # 16,C,17
                                       nn.BatchNorm1d(C),
                                       nn.SELU(),
                                       nn.MaxPooling1D(2, stride=1) # 16,20,16
                                      ) # 
        self.lstm = nn.LSTM(n_features_, hidden_, num_layers=n_layers, batch_first=True) # 
        self.fc = nn.Linear(in_, out_) # 
        
    def forward(self, inputs): # input_seq: sequencces
#         x = self.convblock(x)
#         x, h = self.lstm(x)
#         x = torch.reshape(x, (x.shape[0], x.shape[1]*x.shape[2]))
#         out = self.fc(x)

        
        x = self.convblock(x.view(len(x), 1, -1))

        out = self.convblock(inputs)
        out, (h,c) = self.lstm()

        out, (self.h, self.c) = self.lstm(x.view(len(x), self.seq_len-1, -1), (self.h, self.c))
        out = out.view(self.seq_len-1, len(input_seq))
        out = self.fc(out)
        
        if apply_sigmoid:
            out = F.sigmoid(out, dim=1)
        return out
    
    def init_hidden(self, batch_s):
        self.hc = (torch.zeros(d*num_layers, batch_s, H_out),
                   torch.zeros(d*num_layers, batch_s, H_out))